In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

output_folder = './output_images/'

In [2]:
# Calibrate the Camera

# prepare object points
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    # Load the image
    img = cv2.imread(fname)
    # Grayscale the image
    gray = cv2.cvtColor(img ,cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If chessboard corners were found, add object points, image points to the arrays
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

# Grab the image shape
img_size = (img.shape[1], img.shape[0])

# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size[::-1], None, None)

In [3]:
def saveImage(img, file_name=None, file_name_extension=None, isRGB=True):
    if file_name!=None:
        # Slice out the filename from the path
        new_file_name = file_name[max(file_name.rfind('\\'), file_name.rfind('/')) + 1:file_name.rfind('.')]
        # If the filename should be extended, add the extension
        if file_name_extension!=None:
            new_file_name += file_name_extension
        # Save the image to a file
        if isRGB:
            cv2.imwrite(output_folder + new_file_name + '.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        else:
            cv2.imwrite(output_folder + new_file_name + '.png', img)

In [4]:
# Undistort an image
def undistortImage(img, file_name=None):
    # Save the original image to ourput folder
    saveImage(img, file_name=file_name)
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    # Save the image to the output folder
    saveImage(undistorted, file_name=file_name, file_name_extension='_01_undistorted')
    #Return the undistorted image
    return undistorted

In [5]:
def transformImage(img, src, dst, file_name=None, draw_polygon=(True,False), binarized=False):
    
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size)
    
    if draw_polygon[0]:
        #Copy image in order to draw the polygon
        p_image = img.copy()
        
        # Create the polygon
        polygon = np.array(src, np.int32)
        polygon = polygon.reshape((-1,1,2))
        
        # Draw the polygons into the image
        cv2.polylines(p_image, [polygon], True, (255,0,0), 2)
        
        # Save the image to the output folder
        if binarized:
            saveImage(p_image, file_name=file_name, file_name_extension='_03_binarized_with_polygon', isRGB=False)
        else:
            saveImage(p_image, file_name=file_name, file_name_extension='_01_undistorted_with_polygon')
                   
    if draw_polygon[1]:
        #Copy image in order to draw the polygon
        p_warped = warped.copy()
        
        # Create the polygon
        warped_polygon = np.array(dst, np.int32)
        warped_polygon = warped_polygon.reshape((-1,1,2))
        
        # Draw the polygon into the image
        cv2.polylines(p_warped, [warped_polygon], True, (255,0,0), 2)
        
        # Save the image to the output folder
        if binarized:
            saveImage(p_warped, file_name=file_name, file_name_extension='_04_binarized_warped', isRGB=False)
        else:
            saveImage(p_warped, file_name=file_name, file_name_extension='_02_warped')
    else:
        # Save the image to the output folder
        if binarized:
            saveImage(warped, file_name=file_name, file_name_extension='_04_binarized_warped', isRGB=False)
        else:
            saveImage(warped, file_name=file_name, file_name_extension='_02_warped')
    
    # Return the resulting image and matrix
    return warped, M

In [6]:
def unwarpImage(img):
    
    # Calculate the perspective transform matrix to unwarp the image
    M = cv2.getPerspectiveTransform(dst, src)
    # Unwarp the image using OpenCV warpPerspective()
    unwarped = cv2.warpPerspective(img, M, img_size)
    
    # Return the resulting image and matrix
    return unwarped, M

In [7]:
def binarizeImage(img, file_name=None):
    
    # Define the sobel thresholds
    sobel_thresh_min=20
    sobel_thresh_max=255
    # Define the saturation thresholds
    saturation_thresh_min=120
    saturation_thresh_max=255
    # Define the lightness thresholds (to little lightness will be substracted during daylight since it might be shadows)
    lightness_thresh_min=0
    lightness_thresh_max=30
    min_mean_lightness_considered_daylight=60
    
    # Convert the image to HLS
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # Get the saturation value
    saturation = hls[:,:,2]
    
    # Create a matrix for a binarized output
    binary_output_sa = np.zeros_like(saturation)
    binary_output_sa[(saturation >= saturation_thresh_min) & (saturation <= saturation_thresh_max)] = 255
    
    # Get the lightness
    lightness = hls[:,:,1]
    # Create a matrix for a binarized lightness output
    binary_output_l = np.zeros_like(lightness)
    if np.mean(lightness)>min_mean_lightness_considered_daylight:
        binary_output_l[(lightness >= lightness_thresh_min) & (lightness <= lightness_thresh_max)] = 255
    
    # Convert to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take the derivative in x
    sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a mask of 255's where the scaled gradient magnitude is > thresh_min and < thresh_max
    binary_output_so = np.zeros_like(scaled_sobel)
    binary_output_so[(scaled_sobel >= sobel_thresh_min) & (scaled_sobel <= sobel_thresh_max)] = 255
    
    # Combine the binary outputs
    binary_output = np.zeros_like(binary_output_so)
    binary_output[((binary_output_so > 0) | (binary_output_sa > 0)) & (binary_output_l == 0)] = 255
    
    # Store the binarized image to disc
    saveImage(binary_output, file_name=file_name, file_name_extension='_03_binarized', isRGB=False)
    
    return binary_output

In [8]:
# Define the values for the sliding window
window_width = 50
window_height = 90 # Break image into 8 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),
           max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(warped, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice
    # The start point shound not be at the edge of the image. So we disregard half of the lane offset on both sides of the image
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(lane_offset/2):int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2+int(lane_offset/2)
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):warped.shape[1]-int(lane_offset/2)], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        sliding_window_offset = window_width/2
        # Define left search window
        l_min_index = int(max(l_center+sliding_window_offset-margin,0))
        l_max_index = int(min(l_center+sliding_window_offset+margin,warped.shape[1]))
        # Define right search window
        r_min_index = int(max(r_center+sliding_window_offset-margin,0))
        r_max_index = int(min(r_center+sliding_window_offset+margin,warped.shape[1]))
        
        # Convolve the window into the vertical slice of the image
        left_image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),
                                         l_min_index:l_max_index],
                                  axis=0)
        right_image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),
                                          r_min_index:r_max_index],
                                   axis=0)
        
        # Find the best left centroid by using past left center as a reference
        left_conv_signal = np.convolve(window, left_image_layer)
        # If anything has been found, set the new left center
        if(np.sum(left_conv_signal)>0):
            l_center = np.argmax(left_conv_signal)+l_min_index-sliding_window_offset
        elif level>1:
            # If we already have a guess in witch direction the line is going, go further in this direction
            l_center+=(window_centroids[-1][0]-window_centroids[-2][0])
            
        # Find the best right centroid by using past right center as a reference
        right_conv_signal = np.convolve(window, right_image_layer)
        # If anything has been found, set the new right center
        if(np.sum(right_conv_signal)>0):
            r_center = np.argmax(right_conv_signal)+r_min_index-sliding_window_offset
        elif level>1:
            # If we already have a guess in witch direction the line is going, go further in this direction
            r_center+=(window_centroids[-1][1]-window_centroids[-2][1])
            
        # Add what we found for that layer, or the former/guessed value if we did not find a line
        window_centroids.append((l_center,r_center))

    return window_centroids

In [9]:
# Generate and store the undistorted chess board images to put them into the output folder by calling undistortImage() method
for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img ,cv2.COLOR_BGR2RGB)
    undistortImage(img, fname)

In [10]:
# Define polygon for transformation
xll = 208    # Lower left x value
xlr = 1107   # lower right x value
xul = 595    # upper left x value
xur = 686    # upper right x value
yu = 450     # upper y value

# Define an offset left and right of the polygon
lane_offset = 320

# Calculate the source
src = np.float32([(xll,img_size[1]), (xul,yu), (xur,yu), (xlr,img_size[1])])
# Calculate the destination
dst = np.float32([[lane_offset,
                   img_size[1]],
                  [lane_offset, 0],
                  [img_size[0]-lane_offset, 0],
                  [img_size[0]-lane_offset,
                   img_size[1]]])

In [11]:
# Searches for lane line and displays it in the image. Image must be RGB format!
def findLaneLines(img, file_name=None):
    
    # Undistort the image
    undistorted_image = undistortImage(img, file_name=file_name)
    
    # Transform/warp the image. (Just necessary if the warped image should be stored. Not during the video.)
    if file_name!=None:
        transformed_image, M = transformImage(undistorted_image,
                                              src,
                                              dst,
                                              file_name=file_name,
                                              draw_polygon=(True,False),
                                              binarized=False)
    
    # Binarize the images
    binarized_image = binarizeImage(undistorted_image, file_name=file_name)
    
    # Transform/warp the binarized image
    warped, M = transformImage(binarized_image,
                               src,
                               dst,
                               file_name=file_name,
                               draw_polygon=(False,False),
                               binarized=True)
    
    
    window_centroids = find_window_centroids(warped,
                                             window_width,
                                             window_height,
                                             margin)

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # Add both left and right window pixels together
        zero_channel = np.zeros_like(template) # Create a zero color channel 
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # Make window pixels green
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # Making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # Overlay the orignal road image with window results
        # Save the windowed image
        if file_name!=None:
            saveImage(output, file_name=file_name, file_name_extension='_05_warped_windows', isRGB=False)
    
    # Create lists of the x and y values of the laft and the right line
    lefty = l_points.nonzero()[0]
    leftx = l_points.nonzero()[1]
    righty = r_points.nonzero()[0]
    rightx = r_points.nonzero()[1]
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Calculate all the x values for the left and the right lane in the order of the y values
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create a numpy array for the y values, counting from 0 to image size and back
    y_values = np.concatenate((np.arange(img_size[1]).reshape(-1, 1),
                               np.arange(img_size[1]-1, -1, -1).reshape(-1, 1)),
                              axis=0)
    
    # Create a numpy array for the x values, by going down the left lane line and going up the right lane line
    x_values = np.concatenate((np.array(left_fitx, np.int16).reshape(-1, 1),
                               np.array(right_fitx[::-1], np.int16).reshape(-1, 1)),
                              axis=0)
    
    # Create a polygon representing the lane and fill it with green color
    lane = np.concatenate((x_values, y_values), axis=1)
    warped_lane_img = np.zeros_like(output)
    cv2.fillPoly(warped_lane_img, [lane], (0,255,0))
    
    # Unwarp the polygon image to the original view
    unwarped_image, M = unwarpImage(warped_lane_img)
    
    # Put the warped lane image over the warped binarized image
    warped_result = cv2.addWeighted(cv2.cvtColor(warped, cv2.COLOR_GRAY2BGR), 1, warped_lane_img, 0.3, 0)
    # Store the warped image with the lane shown
    if file_name!=None:
        saveImage(warped_result, file_name=file_name, file_name_extension='_06_warped_result', isRGB=False)
    
    # Put the unwarped lane image over the original (undistorted) image
    result = cv2.addWeighted(cv2.cvtColor(undistorted_image, cv2.COLOR_RGB2BGR), 1, unwarped_image, 0.3, 0)
    
    
    # Calculate the position of the car and the curvature of the lane
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 26/720 # Meters per pixel in y dimension
    xm_per_pix = 3.7/(img_size[0]-lane_offset) # Meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Calculate the position of the car, relative to the lane
    car_position = np.round(((right_fitx[-1]-img_size[0]/2) - (img_size[0]/2-left_fitx[-1])) * 3.7
                            / (right_fitx[-1]-left_fitx[-1]),
                            2)
    
    # Create the text for displaying the car position and curvature
    car_position_str = ''
    if car_position > 0:
        car_position_str = 'left'
    elif car_position < 0:
        car_position_str = 'right'
    else:
        car_position_str = ''
    
    cv2.putText(result,
                'Radius of Curvature = ' + str(np.round((left_curverad+right_curverad)/2,0)) + '(m)',
                (0,50),
                cv2.FONT_HERSHEY_SIMPLEX,
                2,
                color=(255,255,255))
    
    cv2.putText(result,
                'Vehicle is ' + str(abs(car_position)) + 'm ' + car_position_str + ' of center',
                (0,100),
                cv2.FONT_HERSHEY_SIMPLEX,
                2,
                color=(255,255,255))
    
    # Store the final result, including the text
    if file_name!=None:
        saveImage(result, file_name=file_name, file_name_extension='_07_result', isRGB=False)
    
    # Return the unsistorted image with the lane drawn
    return cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

In [12]:
# Make a list of the test images
images = glob.glob('./test_images/*.jpg')

# Run through the test images and find the lane lines
for fname in images:
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    findLaneLines(img, fname)

In [13]:
# Import MoviePy
from moviepy.editor import VideoFileClip

In [14]:
# Read the images of the video and write the processed images to a new wideo file
video_output = output_folder + 'project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
video_clip = clip1.fl_image(findLaneLines)
%time video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video ./output_images/project_video.mp4
[MoviePy] Writing video ./output_images/project_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [02:37<00:00,  8.18it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_images/project_video.mp4 

Wall time: 2min 37s
